In [1]:
INPUT_LIST = "/Users/joregan/Playing/rdapi_subset"

In [2]:
API_FILENAMES = []
with open(INPUT_LIST) as ilist:
    for line in ilist.readlines():
        parts = line.split()
        API_FILENAMES.append(parts[0])

In [3]:
from pathlib import Path
API_PATH = Path("/Users/joregan/Playing/rdapi/api_output/")

In [21]:
LM_TEXT_PATH = Path("/Users/joregan/Playing/rdapi/lm_text/")

In [4]:
from sync_asr.riksdag.riksdag_api import RiksdagAPI

In [ ]:
!pip install mosestokenizer

In [12]:
from mosestokenizer import MosesSentenceSplitter
splitter = MosesSentenceSplitter('sv')

stdbuf was not found; communication with perl may hang due to stdio buffering.


In [27]:
import re
ALPHA = "abcdefghijklmnopqrstuvwxyzåäö"
NUM = "0123456789"
WORDPUNCT = ":-"
ALLOWED = ' ' + ALPHA + NUM + WORDPUNCT

def cleantext(text: str) -> str:
    text = text.lower()
    text = text.replace(": ", " ")
    text = text.replace(", ", " ")
    text = text.replace(" - ", " ")
    text = re.sub("  +", " ", text)
    chars = [x for x in text if x in ALLOWED]
    text = "".join(chars)

    return text

In [28]:
for filename in API_FILENAMES:
    FILE_PATH = API_PATH / filename
    lm_text_file = LM_TEXT_PATH / filename
    api = RiksdagAPI(filename=str(FILE_PATH))
    paras = [x['text'] for x in api.get_paragraphs_with_ids()]
    paras = [p for p in paras if p != ""]
    sents = splitter(paras)
    tmp_rejoin = "\n".join(sents)
    tmp_rejoin = tmp_rejoin.replace(":\ns ", ":s ")
    tmp_rejoin = tmp_rejoin.replace(":\nen ", ":en ")
    tmp_rejoin = tmp_rejoin.replace(":\nt ", ":t ")
    tmp_rejoin = tmp_rejoin.replace(":\n", "\n")
    sents = tmp_rejoin.split("\n")
    with open(str(lm_text_file), "w") as lm:
        for s in sents:
            lm.write(cleantext(s) + "\n")
    